<a href="https://colab.research.google.com/github/denisangelo/Federated_Learning/blob/main/FL_Yes_Division_Dataset_Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
#Importanto Pytorch, Flower,Opacus
!pip install -q flwr[simulation]
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install opacus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
#Importanto as bibliotecas necessárias
from collections import OrderedDict
from typing import Dict, Optional, List, Tuple
import matplotlib.pyplot as plt

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from flwr.common import Metrics
import torchvision.transforms as transforms
import opacus
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager
from torch.optim.adam import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

In [47]:
#Definindo o dispositivo de treinamento
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE} using Pytorch{torch.__version__} and Flower{fl.__version__}")

Training on cpu using Pytorch2.0.1+cu118 and Flower1.4.0


In [48]:
# Definindo os valores das variáveis
BATCH_SIZE = 100
NUM_CLIENTS= 10
optim_lr=0.001
local_epochs=5
num_rounds=10

In [49]:
#Carregando os dados
def load_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = CIFAR10(root="./data", train=True,download=True, transform=transform)
    testset = CIFAR10(root="./data", train=False, download=True, transform=transform)
    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10 # 10% do conjunto para testes
        len_train = len(ds) - len_val
        lengths =[len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True,num_workers=0))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=False,num_workers=0))
        testloader = DataLoader(testset,batch_size=BATCH_SIZE, shuffle=False,num_workers=0)
        num_examples = {"ds_train": len(ds_train)*BATCH_SIZE, "ds_val": len(ds_val)*BATCH_SIZE}

    return trainloaders, valloaders, testloader,num_examples
    
trainloaders, valloaders, testloader,num_examples = load_data()

Files already downloaded and verified
Files already downloaded and verified


In [50]:
# Definindo o Modelo
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [51]:
# Definindo as funções de treino e teste
def train(net, trainloader, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    for _ in range(epochs):
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()

def test(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [52]:
# Funções de atualização de parâmetros do modelo
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [53]:
# Definindo a classe de cliente Virtual
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader, num_example)-> None:
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.num_example = num_example

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

def client_fn(cid) -> FlowerClient:
        net = Net().to(DEVICE)
        trainloader = trainloaders[int(cid)]
        valloader = valloaders[int(cid)]
        num_example = num_examples
        return FlowerClient(cid, net, trainloader, valloader, num_example)

In [54]:
# Realizando a agregação
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [55]:
# Iniciando o treino federado
# Criando a Função de agregação FedAvg
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,#-->Fração de clientes necessários para treinamento
    fraction_evaluate=1.0, #--> Fração de clientes que serão utilizados para teste
    min_fit_clients=NUM_CLIENTS, #--> Número mínimo que serão utlilizados para treinamento
    min_evaluate_clients=2, #--> Número mínimo de clientes que serão realizados para teste
    min_available_clients=NUM_CLIENTS,#--> Número mínimo de clientes que serão utilizados no treinamento
    evaluate_metrics_aggregation_fn=weighted_average,  
)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}
# Inicia a simulação
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-10 16:26:22,014 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)


(launch_and_evaluate pid=8888) [Client 2] evaluate, config: {} [repeated 7x across cluster]


2023-06-10 16:26:31,907	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-10 16:26:34,099 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'memory': 7865037620.0, 'object_store_memory': 3932518809.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'memory': 7865037620.0, 'object_store_memory': 3932518809.0, 'CPU': 2.0}
INFO flwr 2023-06-10 16:26:34,104 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-10 16:26:34,106 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=16229) 2023-06-10 16:26:40.063457: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO flwr 2023-06-10 16:26:47,305 | server.py:277 | Received initial parameters from one random client
INFO:flwr:Received initial parameters from one r

(launch_and_get_parameters pid=16229) [Client 0] get_parameters
(launch_and_fit pid=16229) [Client 1] fit, config: {}


(pid=16230) 2023-06-10 16:27:00.710326: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


...

(launch_and_fit pid=16229) [Client 7] fit, config: {}
(launch_and_evaluate pid=21101) [Client 7] evaluate, config: {} [repeated 3x across cluster]
(launch_and_fit pid=16229) [Client 0] fit, config: {} [repeated 2x across cluster]
(launch_and_fit pid=16229) [Client 3] fit, config: {} [repeated 2x across cluster]
(launch_and_fit pid=16229) [Client 6] fit, config: {} [repeated 2x across cluster]
(launch_and_fit pid=16229) [Client 1] fit, config: {} [repeated 2x across cluster]


DEBUG flwr 2023-06-10 16:52:04,904 | server.py:232 | fit_round 10 received 10 results and 0 failures
DEBUG:flwr:fit_round 10 received 10 results and 0 failures
DEBUG flwr 2023-06-10 16:52:04,939 | server.py:168 | evaluate_round 10: strategy sampled 10 clients (out of 10)
DEBUG:flwr:evaluate_round 10: strategy sampled 10 clients (out of 10)


(launch_and_evaluate pid=16229) [Client 6] evaluate, config: {}
(launch_and_fit pid=21101) [Client 6] evaluate, config: {}
(launch_and_evaluate pid=16229) [Client 5] evaluate, config: {} [repeated 2x across cluster]


DEBUG flwr 2023-06-10 16:52:24,129 | server.py:182 | evaluate_round 10 received 10 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 10 results and 0 failures
INFO flwr 2023-06-10 16:52:24,133 | server.py:147 | FL finished in 1536.8015956529998
INFO:flwr:FL finished in 1536.8015956529998
INFO flwr 2023-06-10 16:52:24,135 | app.py:218 | app_fit: losses_distributed [(1, 0.017888644909858704), (2, 0.015063292264938353), (3, 0.014120519542694093), (4, 0.013502863502502443), (5, 0.012933032262325285), (6, 0.01267439023256302), (7, 0.012449014317989351), (8, 0.012350237369537354), (9, 0.012376043629646304), (10, 0.012428451704978942)]
INFO:flwr:app_fit: losses_distributed [(1, 0.017888644909858704), (2, 0.015063292264938353), (3, 0.014120519542694093), (4, 0.013502863502502443), (5, 0.012933032262325285), (6, 0.01267439023256302), (7, 0.012449014317989351), (8, 0.012350237369537354), (9, 0.012376043629646304), (10, 0.012428451704978942)]
INFO flwr 2023-06-10 16:52:24,138 | app.py:

History (loss, distributed):
	round 1: 0.017888644909858704
	round 2: 0.015063292264938353
	round 3: 0.014120519542694093
	round 4: 0.013502863502502443
	round 5: 0.012933032262325285
	round 6: 0.01267439023256302
	round 7: 0.012449014317989351
	round 8: 0.012350237369537354
	round 9: 0.012376043629646304
	round 10: 0.012428451704978942
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.3554), (2, 0.4536), (3, 0.482), (4, 0.511), (5, 0.5354), (6, 0.5492), (7, 0.5613999999999999), (8, 0.5684), (9, 0.5722), (10, 0.5748000000000001)]}